In [1]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd 

from tensorflow.keras import datasets, layers, models, regularizers, initializers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from skimage.io import imshow
from skimage.transform import rotate
from skimage.filters.edges import convolve

from skimage.io import imshow, imread
from skimage.transform import resize
from skimage.color import rgb2gray

import dill as pickle

np.random.seed(33)

In [2]:
df_test = pd.read_csv('data/test_labels.csv')
df_train = pd.read_csv('data/train_labels.csv')

bw_loaded = np.load('data/bw_images.npz')
X_train= bw_loaded['a']
X_test = bw_loaded['b']

X_test = X_test.reshape(-1,80,60,1)
X_train = X_train.reshape(-1,80,60,1)

In [3]:
# from model_functions import pick_ylabels, multi_index_counts, test_counts_by_cat, train_counts_by_cat, category_codes, class_weights

In [4]:
def pick_ylabels(column):
    y_train = df_train[column].copy().astype('category').cat.codes
    y_test = df_test[column].copy().astype('category').cat.codes
    return (y_train.values, y_test.values)
def multi_index_counts(col, col2):
    counts = df_test.groupby([col, col2]).count().id
    return counts

def category_codes(column):
    _, ytest = pick_ylabels(column)
    cat_codes = {}
    cat_code_list = []
    for i in range(len(df_test[column].value_counts().index)):
        s = i
        t = df_test[column].value_counts().index[i]
        cat_codes[s] = t
#         cat_code_list.append([s, t])

#     for key in sorted(cat_codes):
#         print("%s: %s" % (key, cat_codes[key]))
    return cat_codes

def test_counts_by_cat(column):
    _, ytest = pick_ylabels(column)
    test_counts_dict = {}
    test_counts = []
    for i in range(len(df_test[column].value_counts().index)):
        s = df_test[column].value_counts().values[i]
        t = df_test[column].value_counts().index[i]
        test_counts_dict[t] = s
        test_counts.append([t, s])
#     for i in sorted(test_counts):
#         print("%s: %s" % (test_counts[0], test_counts[1]))
    return test_counts

def train_counts_by_cat(column):
    y_train = pick_ylabels(column)
    train_counts_dict = {}
    train_counts = []
    for i in range(len(df_test[column].value_counts().index)):
        s = df_train[column].value_counts().values[i]
        t = df_train[column].value_counts().index[i]
        train_counts_dict[t] = s
        train_counts.append([t, s])
#     for i in sorted(test_counts):
#         print("%s: %s" % (test_counts[0], test_counts[1]))
    return train_counts

def class_weights(column):
    train_counts = train_counts_by_cat(column)
    counts_list = []
    ratio_list = []
    ratio_dict = {}
    for x in test_counts:
        counts_list.append(x[1])
    #print(counts_list)
    z = sum(counts_list)
    for x in counts_list:
        if np.round(x/z, 3) ==0:
            ratio_list.append(.001)
        else: 
            ratio_list.append(np.round(x/z, 2))
    for k, v in enumerate(ratio_list):
        ratio_dict[k] = v
    return ratio_dict

def model_results(model, column):
    yhat = model.predict(X_test)
    _ , ytest = pick_ylabels(column)
    accuracy = accuracy_score(ytest, yhat)
    print( 'Accuracy Score: ', accuracy )
    recall = recall_score(ytest, yhat, average='weighted')
    print( 'Recall Score: ', recall)
    y_proba = model.predict_proba(X_test)
    wrong_id_list = []
    pred_cat_list = []
    real_cat_list = []
    for row_idx in range(len(ytest)):
        if ytest[row_idx]!=yhat[row_idx]:
            wrong_id_list.append(row_idx)
            pred_cat_list.append(yhat[row_idx])
            real_cat_list.append(ytest[row_idx])

    arr = np.array([ real_cat_list, pred_cat_list])
    arr = arr.transpose()
    wrong_df = pd.DataFrame( arr, index= wrong_id_list, columns = ['actual', 'predicted'] )
    return (accuracy, recall, wrong_df)

### Predict Sub-Categories for Apparel

In [38]:
app_train = df_train[df_train.masterCategory=='Apparel']
app_train_idx = list(app_df_train.index)
X_train_app = X_train[app_train_idx]

app_test = df_test[df_test.masterCategory=='Apparel']
app_test_idx = list(app_df_test.index)
X_test_app = X_test[app_test_idx]

In [42]:
y_train0 = app_train.subCategory.copy().astype('category').cat.codes
y_test0 = app_test.subCategory.copy().astype('category').cat.codes

In [43]:
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train0)
y_test = to_categorical(y_test0)

In [44]:
print(y_train.shape)
print(y_test.shape)

(9194, 6)
(2619, 6)


In [60]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D

In [68]:
cnn_sub = models.Sequential()

In [69]:
#MaxPool reduces dimensionality of each feature
#Dropout to reduce overfitting

cnn_sub.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(80,60,1)))
cnn_sub.add(layers.MaxPooling2D((2, 2)))
cnn_sub.add(Dropout(0.2))

cnn_sub.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
cnn_sub.add(MaxPooling2D(pool_size=(2, 2)))
cnn_sub.add(Dropout(0.25))

cnn_sub.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
# cnn_sub.add(MaxPooling2D(pool_size=(2, 2)))
cnn_sub.add(Dropout(0.4))

cnn_sub.add(Flatten())

cnn_sub.add(Dense(128, activation='relu'))
cnn_sub.add(Dropout(0.3))
cnn_sub.add(Dense(6, activation='softmax'))

In [70]:
cnn_sub.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 78, 58, 32)        320       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 39, 29, 32)        0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 39, 29, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 37, 27, 64)        18496     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 18, 13, 64)        0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 18, 13, 64)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 16, 11, 128)      

In [71]:
cnn_sub.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [80]:
cnn_sub.fit(X_train_app, y_train, epochs=50, validation_data=(X_test_app, y_test), verbose=1)

Train on 9194 samples, validate on 2619 samples
Epoch 1/50
9194/9194 [==============================] - 13s 1ms/sample - loss: 0.1187 - acc: 0.9621 - val_loss: 0.1931 - val_acc: 0.9576
Epoch 2/50
9194/9194 [==============================] - 13s 1ms/sample - loss: 0.1083 - acc: 0.9670 - val_loss: 0.1886 - val_acc: 0.9553
Epoch 3/50
9194/9194 [==============================] - 13s 1ms/sample - loss: 0.1066 - acc: 0.9657 - val_loss: 0.1753 - val_acc: 0.9565
Epoch 4/50
9194/9194 [==============================] - 13s 1ms/sample - loss: 0.1180 - acc: 0.9636 - val_loss: 0.1741 - val_acc: 0.9569
Epoch 5/50
9194/9194 [==============================] - 13s 1ms/sample - loss: 0.1086 - acc: 0.9674 - val_loss: 0.1606 - val_acc: 0.9595
Epoch 6/50
9194/9194 [==============================] - 13s 1ms/sample - loss: 0.1063 - acc: 0.9655 - val_loss: 0.1770 - val_acc: 0.9515
Epoch 7/50
9194/9194 [==============================] - 13s 1ms/sample - loss: 0.1048 - acc: 0.9670 - val_loss: 0.1955 - val_acc: 

In [91]:
# save the fitted model
model_json = cnn_sub.to_json()
with open("cnn_apparel.json", "w") as json_file:
    json_file.write(model_json)

In [74]:
y_true = y_test0

In [76]:
# get the predictions for the test data
predicted_classes = cnn_sub.predict_classes(X_test_app)

# get the indices to be plotted
correct = np.nonzero(predicted_classes==y_true)[0]
incorrect = np.nonzero(predicted_classes!=y_true)[0]

/home/ubuntu/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:56: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  return getattr(obj, method)(*args, **kwds)


In [77]:
predicted_classes

array([1, 0, 5, ..., 0, 5, 1])

In [79]:
from sklearn.metrics import classification_report
target_names = ["Class {}".format(i) for i in range(6)]
print(classification_report(y_true, predicted_classes, target_names=target_names))

              precision    recall  f1-score   support

     Class 0       0.95      0.98      0.96       368
     Class 1       0.96      0.91      0.93       347
     Class 2       0.90      0.55      0.68        96
     Class 3       0.95      0.98      0.96        82
     Class 4       0.98      0.93      0.95       135
     Class 5       0.96      0.99      0.98      1591

    accuracy                           0.96      2619
   macro avg       0.95      0.89      0.91      2619
weighted avg       0.96      0.96      0.96      2619



### Predict Sub-Categories for Accessories

In [89]:
acc_train = df_train[df_train.masterCategory=='Accessories']
acc_train_idx = list(acc_train.index)
X_train_acc = X_train[acc_train_idx]

acc_test = df_test[df_test.masterCategory=='Accessories']
acc_test_idx = list(acc_test.index)
X_test_acc = X_test[acc_test_idx]

y_train_acc0 = acc_train.subCategory.copy().astype('category').cat.codes
y_test_acc0 = acc_test.subCategory.copy().astype('category').cat.codes

from tensorflow.keras.utils import to_categorical
y_train_acc = to_categorical(y_train_acc0)
y_test_acc = to_categorical(y_test_acc0)

print(y_train_acc.shape)
print(y_test_acc.shape)

(5586, 6)
(1656, 6)


In [84]:
cnn_acc = models.Sequential()

In [85]:
#MaxPool reduces dimensionality of each feature
#Dropout to reduce overfitting

cnn_acc.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(80,60,1)))
cnn_acc.add(layers.MaxPooling2D((2, 2)))
cnn_acc.add(Dropout(0.2))

cnn_acc.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
cnn_acc.add(MaxPooling2D(pool_size=(2, 2)))
cnn_acc.add(Dropout(0.25))

cnn_acc.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
cnn_acc.add(Dropout(0.4))


cnn_acc.add(Flatten())

cnn_acc.add(Dense(128, activation='relu'))
cnn_acc.add(Dropout(0.3))
cnn_acc.add(Dense(6, activation='softmax'))

In [86]:
cnn_acc.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 78, 58, 32)        320       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 39, 29, 32)        0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 39, 29, 32)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 37, 27, 64)        18496     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 18, 13, 64)        0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 18, 13, 64)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 16, 11, 128)      

In [87]:
cnn_acc.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [90]:
cnn_acc.fit(X_train_acc, y_train_acc, epochs=10, validation_data=(X_test_acc, y_test_acc), verbose=1)

Train on 5586 samples, validate on 1656 samples
Epoch 1/10
5586/5586 [==============================] - 8s 1ms/sample - loss: 5.3784 - acc: 0.7053 - val_loss: 0.2950 - val_acc: 0.8877
Epoch 2/10
5586/5586 [==============================] - 8s 1ms/sample - loss: 0.3736 - acc: 0.8813 - val_loss: 0.1463 - val_acc: 0.9589
Epoch 3/10
5586/5586 [==============================] - 8s 1ms/sample - loss: 0.2637 - acc: 0.9239 - val_loss: 0.2154 - val_acc: 0.9185
Epoch 4/10
5586/5586 [==============================] - 8s 1ms/sample - loss: 0.2196 - acc: 0.9338 - val_loss: 0.1077 - val_acc: 0.9716
Epoch 5/10
5586/5586 [==============================] - 8s 1ms/sample - loss: 0.1816 - acc: 0.9440 - val_loss: 0.1520 - val_acc: 0.9662
Epoch 6/10
5586/5586 [==============================] - 8s 1ms/sample - loss: 0.1455 - acc: 0.9552 - val_loss: 0.1314 - val_acc: 0.9668
Epoch 7/10
5586/5586 [==============================] - 8s 1ms/sample - loss: 0.1347 - acc: 0.9603 - val_loss: 0.0842 - val_acc: 0.9819
